In [1]:
import logging,os

logging.getLogger('tensorflow').disabled = True

import warnings  
warnings.filterwarnings('ignore')
# warnings.filterwarnings("ignore",category=FutureWarning)
# warnings.filterwarnings("ignore",category=Warning)
# warnings.filterwarnings("ignore",category=RuntimeWarning)
import tensorflow as tf
import numpy as np

# with warnings.catch_warnings():  
#     warnings.filterwarnings("ignore",category=FutureWarning)
#     warnings.filterwarnings("ignore",category=Warning)
#     warnings.filterwarnings("ignore",category=RuntimeWarning)
#     warnings.filterwarnings("ignore", message="numpy.ufunc size changed")
#     import tensorflow as tf
#     import numpy as np

# import tensorflow as tf

tf.get_logger().setLevel(logging.ERROR)
from collections import defaultdict
from reader import batch_review_normalize, PAD_INDEX, START_INDEX,DataReader, get_review_data
from utils import load_glove, get_shape, count_parameters, load_vocabulary, decode_reviews, log_info
from bleu import compute_bleu
from rouge import rouge


<img src="files/mgf_rp.png">

In [2]:
def get_features(self, user_emb, item_emb, num_layers=1):
        with tf.variable_scope('features', reuse=tf.AUTO_REUSE):
            features = tf.concat([user_emb, item_emb], axis=1)
            for layer in range(num_layers):
                w = tf.get_variable('w{}'.format(layer), [2 * self.F, 2 * self.F], initializer=self.weight_initializer)
                b = tf.get_variable('b{}'.format(layer), [2 * self.F], initializer=self.const_initializer)
                features = tf.matmul(features, w) + b
                features = tf.nn.tanh(features, 'h{}'.format(layer))

            return features
        
def build_rating_predictor(self):
    features = self._get_features(self.user_emb, self.item_emb)

    with tf.variable_scope('rating'):
        rating_labels = tf.reshape(self.ratings, [-1, 1])
        rating_preds = self.global_rating + tf.layers.dense(features, units=1, name='prediction')
        self.rating_loss = tf.losses.mean_squared_error(rating_labels, rating_preds)

        self.rating_preds = tf.clip_by_value(rating_preds, clip_value_min=1.0, clip_value_max=5.0)
        self.mae, mae_update = tf.metrics.mean_absolute_error(rating_labels, self.rating_preds, name='metrics/MAE')
        self.rmse, rmse_update = tf.metrics.root_mean_squared_error(rating_labels, self.rating_preds, name='metrics/RMSE')

        metric_vars = tf.get_collection(tf.GraphKeys.LOCAL_VARIABLES, scope="rating/metrics")
        self.init_metrics = tf.variables_initializer(var_list=metric_vars)
        self.update_metrics = tf.group([mae_update, rmse_update])

<img src="files/mgf_att.png">

In [3]:
def attention_layer(self, h, features, features_proj):
        with tf.variable_scope('attention'):
            L = get_shape(features)[1]

            w = tf.get_variable('w', [self.C, self.D], initializer=self.weight_initializer)
            b = tf.get_variable('b', [self.D], initializer=self.const_initializer)
            w_att = tf.get_variable('w_att', [self.D, 1], initializer=self.weight_initializer)
            b_att = tf.get_variable('b_att', [1], initializer=self.const_initializer)

            h_att = tf.nn.tanh(features_proj + tf.expand_dims(tf.matmul(h, w), 1) + b)
            out_att = tf.reshape(tf.matmul(tf.reshape(h_att, [-1, self.D]), w_att) + b_att, [-1, L])
            alpha = tf.nn.softmax(out_att)
            context = tf.reduce_sum(features * tf.expand_dims(alpha, 2), 1, name='context')
            return context, alpha

<img src="files/mgf_fusion.png">

In [4]:
def fusion_gate(self, x, h, s_features, v_features):
        with tf.variable_scope('fusion_gate'):
            w_x = tf.get_variable('w_x', [self.W, 1], initializer=self.weight_initializer)
            w_h = tf.get_variable('w_h', [self.C, 1], initializer=self.weight_initializer)
            b = tf.get_variable('b', [1], initializer=self.const_initializer)
            beta = tf.nn.sigmoid(tf.matmul(x, w_x) + tf.matmul(h, w_h) + b)  # (N, 1)
            weighted_features = tf.multiply(beta, s_features) + tf.multiply((1. - beta), v_features)
            return weighted_features, beta

<img src="files/mgf_nlp.png">

In [5]:
def init_lstm(self):
        with tf.variable_scope('init_lstm'):
            user_item_emb = tf.concat([self.user_emb, self.item_emb], axis=1)

            w_h_ui = tf.get_variable('w_h_ui', [self.D, self.C], initializer=self.weight_initializer)
            b_h = tf.get_variable('b_h', [self.C], initializer=self.const_initializer)
            h = tf.matmul(user_item_emb, w_h_ui) + b_h

            w_c_ui = tf.get_variable('w_c_ui', [self.D, self.C], initializer=self.weight_initializer)
            b_c = tf.get_variable('b_c', [self.C], initializer=self.const_initializer)
            c = tf.matmul(user_item_emb, w_c_ui) + b_c

            h = tf.nn.tanh(h)
            c = tf.nn.tanh(c)
            return c, h

def decode_lstm(self, x, h, context):
        with tf.variable_scope('decode_lstm'):
            w_h = tf.get_variable('w_h', [self.C, self.W], initializer=self.weight_initializer)
            b_h = tf.get_variable('b_h', [self.W], initializer=self.const_initializer)
            w_out = tf.get_variable('w_out', [self.W, self.V], initializer=self.weight_initializer)
            b_out = tf.get_variable('b_out', [self.V], initializer=self.const_initializer)

            h = tf.layers.dropout(h, self.dropout_rate, training=self.is_training)
            h_logits = tf.matmul(h, w_h) + b_h

            w_ctx2out = tf.get_variable('w_ctx2out', [get_shape(context)[1], self.W], initializer=self.weight_initializer)
            h_logits += tf.matmul(context, w_ctx2out)

            h_logits += x
            h_logits = tf.nn.tanh(h_logits)

            h_logits = tf.layers.dropout(h_logits, self.dropout_rate, training=self.is_training)
            out_logits = tf.matmul(h_logits, w_out) + b_out
            return out_logits

def build_review_generator(self):
        with tf.variable_scope('review', reuse=tf.AUTO_REUSE):
            reviews_inputs = self.reviews[:, :self.T - 1]
            reviews_emb = tf.nn.embedding_lookup(self.word_matrix, reviews_inputs)
            reviews_labels = self.reviews[:, 1:]
            mask = tf.to_float(tf.not_equal(reviews_labels, PAD_INDEX))

            loss = 0.0

            self.cell = tf.nn.rnn_cell.BasicLSTMCell(num_units=self.C, name='LSTM_Cell')
            c, h = self._init_lstm()

            for t in range(self.T - 1):
                x = reviews_emb[:, t, :]
                visual_context, alpha = self._attention_layer(h, self.visual_features, self.visual_projection)
                context, beta = self._fusion_gate(x, h, self.sentiment_features, visual_context)
                cell_input = tf.concat([x, context], axis=1)
                _, (c, h) = self.cell(inputs=cell_input, state=[c, h])

                logits = self._decode_lstm(x, h, context)

                loss += tf.reduce_sum(
                  tf.nn.sparse_softmax_cross_entropy_with_logits(labels=reviews_labels[:, t], logits=logits) * mask[:, t])

            self.review_loss = loss / tf.reduce_sum(mask)

def build_review_sampler(self, max_decode_length):
        with tf.variable_scope('review', reuse=tf.AUTO_REUSE):
            sampled_word_list = []
            beta_list = []
            alpha_list = []

            c, h = self._init_lstm()

            batch_size = tf.shape(self.users)[0]
            sampled_word = tf.fill([batch_size], START_INDEX)
            for t in range(max_decode_length):
                x = tf.nn.embedding_lookup(self.word_matrix, sampled_word)

                visual_context, alpha = self._attention_layer(h, self.visual_features, self.visual_projection)
                alpha_list.append(alpha)
                context, beta = self._fusion_gate(x, h, self.sentiment_features, visual_context)
                beta_list.append(beta)

                cell_input = tf.concat([x, context], axis=1)
                _, (c, h) = self.cell(inputs=cell_input, state=[c, h])

                logits = self._decode_lstm(x, h, context)

                sampled_word = tf.argmax(logits, 1)
                sampled_word_list.append(sampled_word)

            self.sampled_reviews = tf.transpose(tf.stack(sampled_word_list), (1, 0))  # (N, max_len)
            self.alphas = tf.transpose(tf.stack(alpha_list), (1, 0, 2))  # (N, T, L)
            self.betas = tf.transpose(tf.squeeze(tf.stack(beta_list), axis=2), (1, 0))  # (N, T)


<img src="files/arch.jpg">

In [6]:
class Model:

    def __init__(self, total_users, total_items, global_rating,
               num_factors, img_dims, vocab_size,
               word_dim, lstm_dim, max_length, dropout_rate):
        
        self.total_users = total_users
        self.total_items = total_items
        self.global_rating = global_rating
        self.dropout_rate = dropout_rate
        self.F = num_factors
        self.L = img_dims[0]
        self.D = img_dims[1]
        self.V = vocab_size
        self.W = word_dim
        self.C = lstm_dim
        self.T = max_length
        Model._get_features = get_features
        Model._build_rating_predictor = build_rating_predictor
        Model._attention_layer = attention_layer
        Model._fusion_gate = fusion_gate
        Model._init_lstm = init_lstm
        Model._decode_lstm = decode_lstm
        Model._build_review_generator = build_review_generator
        Model._build_review_sampler = build_review_sampler
        
        
        self.weight_initializer = tf.contrib.layers.xavier_initializer()
        self.const_initializer = tf.zeros_initializer()

        self.users = tf.placeholder(tf.int32, shape=[None])
        self.items = tf.placeholder(tf.int32, shape=[None])
        self.ratings = tf.placeholder(tf.float32, shape=[None])
        self.images = tf.placeholder(tf.float32, shape=[None, self.L, self.D])
        self.reviews = tf.placeholder(tf.int32, shape=[None, None])
        self.is_training = tf.placeholder(tf.bool)

        self._init_embeddings()

        self.user_emb = tf.nn.embedding_lookup(self.user_matrix, self.users)
        self.item_emb = tf.nn.embedding_lookup(self.item_matrix, self.items)

        self.sentiment_features = self._get_features(self.user_emb, self.item_emb)
        self.sentiment_features = self._batch_norm(self.sentiment_features, name='review/sentiment')
        self.visual_features = self._batch_norm(self.images, name='review/visual')
        self.visual_projection = self._visual_projection(self.visual_features)

        self._build_rating_predictor()
        self._build_review_generator()
        self._build_review_sampler(max_decode_length=self.T)

    def _init_embeddings(self):
        self.user_matrix = tf.get_variable(
          name='user_matrix',
          shape=[self.total_users, self.F],
          initializer=self.weight_initializer,
          dtype=tf.float32
        )

        self.item_matrix = tf.get_variable(
          name='item_matrix',
          shape=[self.total_items, self.F],
          initializer=self.weight_initializer,
          dtype=tf.float32
        )

        self.word_matrix = tf.get_variable(
          name='word_matrix',
          shape=[self.V, self.W],
          initializer=tf.constant_initializer(load_glove(self.V, self.W)),
          dtype=tf.float32
        )

    def _batch_norm(self, x, name=None):
        return tf.contrib.layers.batch_norm(inputs=x,
                                            decay=0.95,
                                            center=True,
                                            scale=True,
                                            is_training=self.is_training,
                                            updates_collections=None,
                                            scope=(name + '_batch_norm'))
    

    def _visual_projection(self, features):
        with tf.variable_scope('review/visual_projection'):
            w = tf.get_variable('w', [self.D, self.D], initializer=self.weight_initializer)
            features_flat = tf.reshape(features, [-1, self.D])
            features_proj = tf.matmul(features_flat, w)
            features_proj = tf.reshape(features_proj, [-1, self.L, self.D])
            return features_proj

    


    
    def feed_dict(self, users, items, ratings=None, images=None, reviews=None, is_training=False):
        fd = {
          self.users: users,
          self.items: items,
          self.is_training: is_training
        }
        if ratings is not None:
            fd[self.ratings] = ratings
        if images is not None:
            fd[self.images] = images
        if reviews is not None:
            fd[self.reviews] = batch_review_normalize(reviews, self.T)

        return fd


In [7]:
tf.app.flags.DEFINE_string('f', '', 'kernel')

tf.flags.DEFINE_string("data_dir", "data",
                       """Path to the data directory""")

tf.flags.DEFINE_float("learning_rate", 0.001,
                      """Learning rate (default: 3e-4)""")
tf.flags.DEFINE_float("dropout_rate", 0.2,
                      """Probability of dropping neurons (default: 0.2)""")
tf.flags.DEFINE_float("lambda_reg", 1e-4,
                      """Lambda hyper-parameter for regularization (default: 1e-4)""")

tf.flags.DEFINE_integer("num_epochs", 20,
                        """Number of training epochs (default: 20)""")
tf.flags.DEFINE_integer("batch_size", 256,
                        """Batch size of reviews (default: 64)""")
tf.flags.DEFINE_integer("num_factors", 256,
                        """Number of latent factors for users/items (default: 256)""")
tf.flags.DEFINE_integer("word_dim", 200,
                        """Word embedding dimensions (default: 200)""")
tf.flags.DEFINE_integer("lstm_dim", 256,
                        """LSTM hidden state dimensions (default: 256)""")
tf.flags.DEFINE_integer("max_length", 20,
                        """Maximum length of reviews to be generated (default: 20)""")
tf.flags.DEFINE_integer("display_step", 10,
                        """Display info after number of steps (default: 10)""")

tf.flags.DEFINE_boolean("allow_soft_placement", True,
                        """Allow device soft device placement""")


FLAGS = tf.flags.FLAGS

In [8]:
def check_scope_rating(var_name):
    for name in ['user', 'item', 'features', 'rating']:
        if name in var_name:
            return True
    return False


def check_scope_review(var_name):
    for name in ['user', 'item', 'features', 'review']:
        if name in var_name:
            return True
    return False

def train_fn(model):
    global_step = tf.train.create_global_step()

    trainable_vars = tf.trainable_variables()
    count_parameters(trainable_vars)

    optimizer = tf.train.AdamOptimizer(FLAGS.learning_rate)

    rating_l2_loss = tf.add_n([tf.nn.l2_loss(v) for v in trainable_vars
                                 if check_scope_rating(v.name) and 'bias' not in v.name])
    model.rating_loss = model.rating_loss + FLAGS.lambda_reg * rating_l2_loss

    review_l2_loss = tf.add_n([tf.nn.l2_loss(v) for v in trainable_vars
                                 if check_scope_review(v.name) and 'bias' not in v.name])
    model.review_loss = model.review_loss + FLAGS.lambda_reg * review_l2_loss

    update_rating = optimizer.minimize(model.rating_loss, name='update_rating', global_step=global_step)
    update_review = optimizer.minimize(model.review_loss, name='update_review')

    return update_rating, update_review, global_step

In [9]:
vocab = load_vocabulary(FLAGS.data_dir)
data_reader = DataReader(FLAGS.data_dir)

model = Model(total_users=data_reader.total_users, total_items=data_reader.total_items,
                global_rating=data_reader.global_rating, num_factors=FLAGS.num_factors,
                img_dims=[196, 512], vocab_size=len(vocab), word_dim=FLAGS.word_dim,
                lstm_dim=FLAGS.lstm_dim, max_length=FLAGS.max_length, dropout_rate=FLAGS.dropout_rate)


Reading vocabulary: data/vocab.pkl
Reading data: data/users.txt
Reading data: data/items.txt
Total users: 2909, total items: 2726
Reading data: data/train.pkl
Reading data: data/test.pkl


  2%|▏         | 4/220 [00:00<00:06, 34.76it/s]

Global rating: 3.97
Reading image id_to_idx: data/train.id_to_idx.pkl
Reading image features: data/img_feats/train


  7%|▋         | 4/55 [00:00<00:01, 39.10it/s]

Reading image id_to_idx: data/test.id_to_idx.pkl
Reading image features: data/img_feats/test


100%|██████████| 55/55 [00:01<00:00, 37.29it/s]


Loading pre-trained word embeddings
Total 400000 word vectors in Glove 6B 200d.
Number of OOV words: 915


In [10]:
update_rating, update_review, global_step = train_fn(model)

user_matrix:0                                                                        744704 params
----------------------------------------------------------------------------------------------------
item_matrix:0                                                                        697856 params
----------------------------------------------------------------------------------------------------
word_matrix:0                                                                       2669200 params
----------------------------------------------------------------------------------------------------
features/w0:0                                                                        262144 params
----------------------------------------------------------------------------------------------------
features/b0:0                                                                           512 params
----------------------------------------------------------------------------------------------------


In [12]:
def test_run(sess,log_file,batch_size=64):
        review_gen_corpus = defaultdict(list)
        review_ref_corpus = defaultdict(list)

        photo_bleu_scores = defaultdict(list)
        photo_rouge_scores = defaultdict(list)

        review_bleu_scores = defaultdict(list)
        review_rouge_scores = defaultdict(list)
        
        sess.run(model.init_metrics)
        test_step = 0
        for users, items, ratings in data_reader.read_test_set(batch_size, rating_only=True):
            test_step += 1

            fd = model.feed_dict(users, items, ratings)
            sess.run(model.update_metrics, feed_dict=fd)

            review_users, review_items, review_ratings, photo_ids, reviews = get_review_data(users, items, ratings,
                                                                                             data_reader.test_review)
            img_idx = [data_reader.test_id2idx[photo_id] for photo_id in photo_ids]
            images = data_reader.test_img_features[img_idx]

            fd = model.feed_dict(users=review_users, items=review_items, images=images)
            _reviews, _alphas, _betas = sess.run([model.sampled_reviews, model.alphas, model.betas], feed_dict=fd)

            gen_reviews = decode_reviews(_reviews, vocab)
            ref_reviews = [decode_reviews(batch_review_normalize(ref), vocab) for ref in reviews]

            for user, item, gen, refs,ratings,photo in zip(review_users, review_items, gen_reviews, ref_reviews,review_ratings,photo_ids):
#                 print("review_users",user)
#                 print("review_items",item)
#                 print("ref_reviews",refs)
#                 print("gen_reviews",gen)
#                 print("rating",ratings)
#                 print("photo_id",photo)
                review_gen_corpus[(user, item)].append(gen)
                review_ref_corpus[(user, item)] += refs

                bleu_scores = compute_bleu([refs], [gen], max_order=4, smooth=True)
                for order, score in bleu_scores.items():
                    photo_bleu_scores[order].append(score)

                rouge_scores = rouge([gen], refs)
                for metric, score in rouge_scores.items():
                    photo_rouge_scores[metric].append(score)
            
            break;
                    
        _mae, _rmse = sess.run([model.mae, model.rmse])
        log_info(log_file, '\nRating prediction results: MAE={:.3f}, RMSE={:.3f}'.format(_mae, _rmse))
        log_info(log_file, '\nReview generation results:')
        log_info(log_file, '- Photo level: BLEU-scores = {:.2f}, {:.2f}, {:.2f}, {:.2f}'.format(
            np.array(photo_bleu_scores[1]).mean() * 100, np.array(photo_bleu_scores[2]).mean() * 100,
            np.array(photo_bleu_scores[3]).mean() * 100, np.array(photo_bleu_scores[4]).mean() * 100))
        
        for user_item, gen_reviews in review_gen_corpus.items():
            references = [list(ref) for ref in set(tuple(ref) for ref in review_ref_corpus[user_item])]
            user_item_bleu_scores = defaultdict(list)
            for gen in gen_reviews:
#                 print("user_item",user_item)
#                 print("references",references)
#                 print("gen",gen)
                bleu_scores = compute_bleu([references], [gen], max_order=4, smooth=True)
                for order, score in bleu_scores.items():
                    user_item_bleu_scores[order].append(score)
            for order, scores in user_item_bleu_scores.items():
                  review_bleu_scores[order].append(np.array(scores).mean())

            user_item_rouge_scores = defaultdict(list)
            for gen in gen_reviews:
                rouge_scores = rouge([gen], references)
                for metric, score in rouge_scores.items():
                    user_item_rouge_scores[metric].append(score)
            for metric, scores in user_item_rouge_scores.items():
                review_rouge_scores[metric].append(np.array(scores).mean())
                
        log_info(log_file, '- Review level: BLEU-scores = {:.2f}, {:.2f}, {:.2f}, {:.2f}'.format(
            np.array(review_bleu_scores[1]).mean() * 100, np.array(review_bleu_scores[2]).mean() * 100,
            np.array(review_bleu_scores[3]).mean() * 100, np.array(review_bleu_scores[4]).mean() * 100))
        
        for metric in ['rouge_1', 'rouge_2', 'rouge_l']:
            log_info(log_file, '- Photo level: {} = {:.2f}, {:.2f}, {:.2f}'.format(
              metric,
              np.array(photo_rouge_scores['{}/p_score'.format(metric)]).mean() * 100,
              np.array(photo_rouge_scores['{}/r_score'.format(metric)]).mean() * 100,
              np.array(photo_rouge_scores['{}/f_score'.format(metric)]).mean() * 100))
            log_info(log_file, '- Review level: {} = {:.2f}, {:.2f}, {:.2f}'.format(
              metric,
              np.array(review_rouge_scores['{}/p_score'.format(metric)]).mean() * 100,
              np.array(review_rouge_scores['{}/r_score'.format(metric)]).mean() * 100,
              np.array(review_rouge_scores['{}/f_score'.format(metric)]).mean() * 100))

        log_info(log_file, '')


In [13]:
log_file = open('log.txt', 'w')
config = tf.ConfigProto(allow_soft_placement=FLAGS.allow_soft_placement)
config.gpu_options.allow_growth = True
with tf.Session(config=config) as sess:
    saver = tf.train.Saver()
    ckpt = tf.train.get_checkpoint_state('./model_dir')
    if ckpt:
        print("Reading model parameters from %s".format(ckpt.model_checkpoint_path))
        saver.restore(sess, ckpt.model_checkpoint_path)
    else:
        print("Creating model with fresh parameters.")
        sess.run(tf.global_variables_initializer())
    for epoch in range(1, 5 + 1):
        log_info(log_file, "\nEpoch: {}/{}".format(epoch, FLAGS.num_epochs))
        count = 0
        sum_rating_loss = 0
        sum_review_loss = 0
        for users, items, ratings in data_reader.read_train_set(FLAGS.batch_size, rating_only=True):
            count += 1
            fd = model.feed_dict(users=users, items=items, ratings=ratings, is_training=True)
            _step, _, _rating_loss = sess.run([global_step, update_rating, model.rating_loss], feed_dict=fd)
            sum_rating_loss += _rating_loss

            review_users, review_items, _, photo_ids, reviews = get_review_data(users, items, ratings,
                                                                                data_reader.train_review)
            img_idx = [data_reader.train_id2idx[photo_id] for photo_id in photo_ids]
            images = data_reader.train_img_features[img_idx]

            fd = model.feed_dict(users=review_users, items=review_items, images=images,
                                 reviews=reviews, is_training=True)
            _, _review_loss = sess.run([update_review, model.review_loss], feed_dict=fd)
            sum_review_loss += _review_loss

            if _step % FLAGS.display_step == 0:
                data_reader.iter.set_postfix(rating_loss=(sum_rating_loss / count),review_loss=(sum_review_loss / count))
        
        checkpoint_path = os.path.join('./model_dir', "mrg.ckpt")
        saver.save(sess, checkpoint_path)
        
        # Testing
        

Reading model parameters from %s


Training:   0%|          | 0/55 [00:00<?, ?it/s]


Epoch: 1/20


Training:   0%|          | 0/55 [00:00<?, ?it/s]


Epoch: 2/20


...

Training:   0%|          | 0/55 [00:00<?, ?it/s]


Epoch: 4/20


Training:   0%|          | 0/55 [00:00<?, ?it/s]


Epoch: 5/20


Training: 100%|██████████| 55/55 [01:15<00:00,  1.38s/it, rating_loss=0.521, review_loss=4.4] 


In [45]:
log_file = open('log.txt', 'w')
with tf.Session(config=config) as sess:
    saver = tf.train.Saver()
    ckpt = tf.train.get_checkpoint_state('./model_dir')
    if ckpt:
        print("Reading model parameters from %s".format(ckpt.model_checkpoint_path))
        saver.restore(sess, ckpt.model_checkpoint_path)
    else:
        print("No Checkpoint Found Exiting")
    test_run(sess,log_file,256)

Reading model parameters from %s















Testing:   0%|          | 0/1751 [00:00<?, ?it/s]


Rating prediction results: MAE=1.651, RMSE=1.657

Review generation results:
- Photo level: BLEU-scores = 27.64, 16.27, 14.30, 13.90
user_item (907, 754)
references [['their', 'macarons', '?'], ['we', 'had', 'four', 'flavors', ':', 'lemon', ',', 'raspberry', ',', 'coffee', ',', 'and', 'chocolate', '.'], ['they', 'neither', 'smelled', 'nor', 'tasted', 'like', 'chocolate', 'and', 'coffee', '.'], ['the', 'lemon', 'and', 'raspberry', 'flavors', 'were', 'on', 'par', '.'], ['i', 'could', 'not', 'say', 'the', 'same', 'for', 'the', 'coffee', 'and', 'chocolate', 'flavors', 'though', '.']]
gen ['the', 'chocolate', 'cake', 'was', 'a', 'little', 'too', 'much', '.']
user_item (735, 1490)
references [['while', 'we', 'waited', 'for', 'pizza', ',', 'we', '<UNK>', 'on', 'some', 'delicious', 'appetizers', '.'], ['pizza', 'was', 'delicious', 'even', 'leftovers', 'reheated', 'a', 'few', 'days', 'later', '.']]
gen ['i', 'ordered', 'the', 'chicken', 'and', 'chicken', 'sandwich', '.']
- Review level: BLEU-s

# 90% of this code is from https://github.com/PreferredAI/mrg